# Short-Read Aligment

## Introduction

DNA is the informational molecule that contains instructions for making and regulating proteins and eventually traits expressed by cells. This information is stored in sequences of nucleotides referred to as bases or base pairs (bp). There are four possible bases: adenine (A), thymine (T), guanine (G), and cytosine (C). During sequencing, sample DNA is broken into many pieces called reads whose sequences are recorded in parallel. After that, a computer must put the pieces back in order by aligning matching sequences at the ends of the pieces with a reference genome. Programs that perform this function are called aligners. 
   
It is not uncommon to have thousands and even millions of reads that need to be aligned, and genomes contain massive amounts of data. The human genome, for example, is over 3 billion base pairs. Managing memory and, ultimately speeding this process up, is the main challenges when developing alignment tools. 

The problem itself is embarassingly parallel as each read can be aligned independently. This process has three basic steps:
1. Input read and genome information
2. Align read to genome
3. Output alignment

Things get a little sticky in steps one and three, and we has to watch out for inefficiencies and race conditions, respectively. 

## Objectives

I am a studying genetics and biochemistry and eventually I would like to go into bioinformatics, which is the use of computer science and math to organize and analyze biological data. As such, I chose a project in the domain of genomics and divided it into two parts. 

In part one, I looked at an existing alignment software called Bowtie2 and had two objectives:

* Set up a working example
* Identify strategies used to speed up alignment

In part two, I wanted to get a better understanding of the parallelization methods learned in class, so I decide to write my own  with two objectives also:

* Write a working program that could align short reads to a larger genome
* Speed it up through parallelization

## Part 1 - Bowtie

Bowtie2 is a short read alignment tool that is used to align sequences to reference genomes often used in genomics research pipelines. It is a staple in many assembly and annotation workflows. Note that the program is bowtie2 even though the naming convention on the HPCC refers to it as bowtie. Though Bowtie is the older verision of Bowtie2, they are different with different uses, syntax, . Bowtie is for reads shorter than 50 base pairs and only supports ungapped mapping. Bowtie2 allows for gapping in its mapping and higher sensitivity to reads longer than 50bp compared to bowtie. It's not the end of the world if you use the wrong one, but it has the potential to give you funny results depending on what you're looking at.

#### Objective 1 - Run an Example

Bowtie2 is already installed on the HPCC. To load the example code, we will be using getexample from the powertools module.

1. Load the powertools module by running the following line:
module load powertools
2. Download the bowtie2 example
getexample bowtie
3. Run the example
qsub bowtie.qsub

#### Objective 2 - Useful Strategies

Bowtie2 runs in parallel and supports different options for parallelization. It has options for both multi-processing and multi-threading. Multi-threading is more advantageous as the large genome index and read data are not duplicated unecessarily. Bowtie2 also uses a queuing lock to organize threads waiting to access critical sections and data. Queuing locks work similarly to spin locks in that each thread will sleep as it waits, continuously probing until it knows it has access. However, each thread probes an entry on its own cache line, reducing the amount of messages passed between threads. To speed up writing, bowtie2 generates multiple output files that are concatenated together. It also uses index-aided alignment to to reduce the search space within the genome.

However, these benefits are lessened with gapped alignments. Gapped alignments are necessitated by missing or extra bases in the sequence due to either sequencing errors or true insertion/deletion mutations in the sample that differ from the reference genome. Bowtie generally will not align reads spanning these gaps and thus data can be thrown out. Bowtie2 differs from Bowtie in that it uses two stages to allow for gapped alignment. It has an initial, ungapped seed-searching stage and a secondary gapped extension stage that utilizes parallel processing to fully align the whole read accounting for any gaps. 


## Part 2 - ClipOn

I chose to use OpenMP in order to reduce copies of my reads and genome cluttering up precious memory as I learned in Part 1. I chose to write my program in C because I wanted to get more comfortable using it as it gives the programmer a lot more control than python. But, quite honestly, I picked between C and C++ arbitrarily, and, looking back, I should have wrote this in C++. In C, strings are not their own data type, rather they are an array of characters and are a lot easier to mess up as managing memory allocation is on you. In C++, strings are  their own objects in the standard library with their own memory management and control. Additionally, and this would have been the most helpful, they also come with built in functions for string manipulation. As a bulk of the operation is reading and managing strings, this would have saved a lot of time and headache.

My priority was getting this running, so, in order to minimize my time fighting FASTA formatting and string manipulation, I made a dummy genome to get the program working. I took the first million digits of pi and converted them to A, T, C, or G depending on how many times each of them divided by three using floor division. I made the reads by going to five random spots in the genome and then collecting a random number of bases after that between 1 and 1000. 

Very generally, ClipOn works by reading in the reads, getting the first ten bases, and then finding where those ten bases are in the reference genome. The parallel and serial versions both do this slightly differently. As in Bowtie, ClipOn does not do paired-end sequencing; it makes its alignment only using the first ten bases of the read. For my dummy genome this is perfect for our 1 million bp genome. If each base has a random 1 in 4 chance of ocurring, then the odds of a ten-base sequence randomly occuring is $1/4^{10} $ or one in roughly a million. 

#### Part 1 - Serial

To make a more fair comparison, I tried to write the most efficient serial program I could with my limited knowledge of C. I combined steps one and two of the basic workflow by reading in the reads and aligning them in the same step. The program reads in the read, gets the first ten bases, and then outputs the index of the starting position. This ended up being faster than the serialized version of Clipon-parallel by about a second for reasons I'll get into in a moment. I recorded the real time, timing the program a minimum of three times or until the standard deviation was less than 1% of the mean. I ran all timing studies on dev-intel14.

<center><b>Serial Time</b></center>

|Iteration|Time(s)|
|---------|:------:|
|1|47.557|
|2|47.331|
|3|47.416|
|||
|Mean|47.434|
|Std|0.114|

#### Part 1 - Parallel

To convert this to parallel, I divided the reading and aligning into two separate steps. This created quite a bit of overhead that is overcome by the further parallelizations but slows down the serial version. I chose to do this as I couldn't figure out how to direct each thread to only one line of the input file. First the program reads through the input reads file and writes each read to an array. Then it splits into one thread for each read and each thread aligns its read, writing the output to an array where the index position corresponds to the line of the file where the read came from. I recorded the time again and observed a 2x speed up from this change alone. 
<center><b>Parallel Time</b></center>

|Iteration|Time(s)|
|---------|:------:|
|1|23.545|
|2|23.637|
|3|23.509|
|||
|Mean|23.564|
|Std|0.066|

Initially, I had a critical section protecting the output array, but realized that, by having each thread access only its own reads' index postion, I could remove it. I ran it for 45 iterations and observed a decrease in time.

<center><b>Parallel Time - No Critical </b></center>

|Mean(s)|Std|
|---------|:------:|
|21.564|0.437|




## Conclusion

#### Future Directions

I know that I can further increase the parallelization of ClipOn. If I had had more time, I would have written a third version of it where I include another section that breaks the reference genome into pieces to search them in parallel. This should, in theory, speed up this process significantly. Additionally, I would have liked to structure my input and output data to match real-world structures so that I could eventually test it on real data. 

#### Outcomes
I was able to create a rudimentary alignment program and speed it up using OMP as we learned in class. Overall, I feel much more comfortable using C now. Being forced to learn about memory allocation was a very good lesson, though it was very frustrating in the moment. Additionally, I feel a lot more confident with parallel programming concepts now that I have been able to wrestle with them in a context and program I understand. Though  ClipOn is admittedly an inelegant version of a short read aligner, in the process of planning, making, and testing it I gained a deeper understanding of the underlying issues and important features of these tools.

# Honors Option: The Wave Equation

## Introduction

#### The Wave Equation

The wave equation is a second-order partial differential equation tha allows us to model how waves move. This has applications in many fields, giving us visualizations of energy moving through a medium. The wave is split into a bunch of points and the velocity of each point is determined by its neighbors. 

#### Numba

Numba is a python package that translates python to optimized machine code at run time. It also automatically runs loops using NumPy arrays on multiple cores, allowing us to use multiprocessing in python. 

## Methods

I used my code from homework one which was already converted into functions for benchmarking purposes. We find the left and right velocity for each point using finite difference and again use finite difference to get a point's acceleration from them. The simplified equation is thus: 
$$\ddot{y}[i] = \frac{y[i+1]-2y[i]+y[i-1]}{dx^2}$$ 

I timed this code on my own machine due to permission issues with numba on the HPCC and got the following results after 10 iterations.

|Serial(s)|Parallel(s)|
|---------|:------:|
|26.14|1.82|


## Conclusions

It is important for us to understand the tools we are using. Though the Numba time is significantly better, an issue with using numba is that it is unclear what timing benefits are conferred by parallelization and what are conferred by compiler optimization as it does both. However, we can see that the functions successfully model the wave equation and are sped up by using Numba.

<img src="./my_awesome.gif" width=50%>


# Sources

Langmead, B., & Salzberg, S. L. (2012). Fast gapped-read alignment with Bowtie 2. *Nature methods*, 9(4), 357-359.

Langmead, B., Wilks, C., Antonescu, V., & Charles, R. (2019). Scaling read aligners to hundreds of threads on general-purpose processors. *Bioinformatics*, 35(3), 421-432.

Wave equation. Brilliant Math &amp; Science Wiki. (n.d.). Retrieved April 30, 2023, from https://brilliant.org/wiki/wave-equation/

Lam, S. K., Pitrou, A., &amp; Seibert, S. (2015). Numba: a LLVM-based Python JIT compiler. *Proceedings of the Second Workshop on the LLVM Compiler Infrastructure in HPC*, 1–6. https://doi.org/10.1145/2833157.2833162 